https://www.data.go.kr/data/15058747/openapi.do

https://www.code.go.kr/stdcode/regCodeL.do

https://shiningyouandme.tistory.com/11

In [135]:
import pandas as pd
from general_functions import get_file_contents


filename = 'ServiceKey 국토교통부 아파트매매 실거래.txt'
api_key = get_file_contents(filename)

In [195]:
url = 'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade'
params ={'serviceKey' : api_key, 'LAWD_CD' : '11110', 'DEAL_YMD' : '202204' }

response = requests.get(url, params=params).text


In [137]:
import requests
import bs4 as bs
import urllib.request

soup = bs.BeautifulSoup(response,'xml')

soup

In [138]:
import xmltodict
xmltodict.parse(response)

OrderedDict([('response',
              OrderedDict([('header',
                            OrderedDict([('resultCode', '00'),
                                         ('resultMsg', 'NORMAL SERVICE.')])),
                           ('body',
                            OrderedDict([('items',
                                          OrderedDict([('item',
                                                        [OrderedDict([('거래금액',
                                                                       '185,000'),
                                                                      ('거래유형',
                                                                       '중개거래'),
                                                                      ('건축년도',
                                                                       '2004'),
                                                                      ('년',
                                                                       '2022'),
        

In [139]:
def load_xml_to_dataframe(response_xml, keyword):
    import urllib.request

    import bs4 as bs
    import pandas as pd

    soup = bs.BeautifulSoup(response_xml, 'xml')

    rows = soup.find_all(keyword)
    columns = rows[0].find_all()

    rowList = []
    nameList = []
    columnList = []

    rowsLen = len(rows)
    columnsLen = len(columns)

    for i in range(0, rowsLen):
        columns = rows[i].find_all()

        for j in range(0, columnsLen):
            if i == 0:
                nameList.append(columns[j].name)  # header
            eachColumn = columns[j].text  # value
            columnList.append(eachColumn)
        rowList.append(columnList)
        columnList = []    # 다음 row의 값을 넣기 위해 비워준다

    result = pd.DataFrame(rowList, columns=nameList)
    return result


In [140]:
df = load_xml_to_dataframe(response, 'item')
df.head()

,거래금액,거래유형,건축년도,년,법정동,아파트,월,일,전용면적,중개사소재지,지번,지역코드,층,해제사유발생일,해제여부
0,"185,000",중개거래,2004,2022,내수동,경희궁의아침4단지,4,14,124.17,서울 종로구,73,11110,14,,
1,"108,000",중개거래,2006,2022,인의동,효성쥬얼리시티,4,9,92.81,서울 종로구,48-2,11110,9,,
2,"34,800",직거래,2021,2022,효제동,이지마루종로,4,11,25.55,,126-2,11110,5,,
3,"17,700",직거래,2017,2022,효제동,포레스트힐시티,4,20,14.6825,,65-2,11110,8,,
4,"18,000",직거래,2014,2022,연건동,이화에수풀,4,26,16.98,,195-10,11110,5,22.05.12,O


In [196]:
df = pd.DataFrame()
for ymd in ['202101', '202204', '202205']:
        params = {'serviceKey': api_key, 'LAWD_CD': '11110', 'DEAL_YMD': ymd}
        d = load_xml_to_dataframe(requests.get(url,
                                                 params=params
                                                 ).text, 'item')
        df =df.append(d, ignore_index = True)



In [197]:
df.shape

(79, 15)

df.info()

In [200]:
for col in ['거래금액', '건축년도', '년', '층']:
    
    #df[col] = df[col].replace(",", "").astype(float, errors = 'raise')
    df[col] = pd.to_numeric(df[col].str.replace(",", ""))

In [201]:
df.head()

,거래금액,거래유형,건축년도,년,법정동,아파트,월,일,전용면적,중개사소재지,지번,지역코드,층,해제사유발생일,해제여부
0,130000,,2000,2021,청운동,청운현대,1,14,129.76,,56-45,11110,2,,
1,150000,,2008,2021,사직동,광화문스페이스본(106동),1,7,144.52,,9-1,11110,6,,
2,175000,,2004,2021,내수동,경희궁의아침2단지,1,15,174.55,,71,11110,4,,
3,175000,,2004,2021,내수동,경희궁의아침2단지,1,15,174.55,,71,11110,4,21.04.08,O
4,180000,,2004,2021,내수동,경희궁의아침3단지,1,17,123.13,,72,11110,13,21.04.19,O


In [202]:
import numpy as np
pd.pivot_table(data=df, index='법정동', columns='거래유형', values='거래금액', aggfunc=sum)

거래유형,,중개거래,직거래
법정동,,,
견지동,127000.0,NaN,NaN
교북동,114000.0,114000.0,NaN
내수동,710000.0,185000.0,200100.0
명륜1가,63000.0,18500.0,NaN
명륜2가,430000.0,NaN,78000.0
무악동,1231730.0,517490.0,NaN
사직동,150000.0,NaN,NaN
숭인동,400550.0,305500.0,10000.0
연건동,NaN,24000.0,18000.0
